In [12]:
!ollama pull llama3


pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest 
pulling 6a0746a1ec1a... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.7 GB                         
pulling 4fa551d4f938... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  12 KB                         
pulling 8ab4849b038c... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  254 B                         
pulling 577073ffcc6c... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  110 B                         
pulling 3f8eb4da87fa... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
!ollama list


NAME             ID              SIZE      MODIFIED       
llama3:latest    365c0bd3c000    4.7 GB    41 seconds ago    


In [19]:
import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext
import requests
from docx import Document
import PyPDF2
import ttkbootstrap as tb

# ========== Ollama Configuration ==========
OLLAMA_URL = "http://localhost:11434/v1/chat/completions"  # Ollama API URL to send chat requests

# Function to check if Ollama is running by sending a GET request to its local server
def check_ollama_running():
    try:
        response = requests.get("http://localhost:11434")  # Ping Ollama server
        return response.status_code == 200  # Return True if server is reachable (status code 200)
    except requests.ConnectionError:  # If Ollama isn't running, handle the error
        return False

# Function to send a prompt to the Ollama model and get a response
def ask_ollama(prompt):
    headers = {"Content-Type": "application/json"}  # Setting up request headers
    data = {
        "model": "llama3",  # Using the Llama3 model
        "messages": [{"role": "user", "content": prompt}],  # User message to Ollama
        "temperature": 0.7  # Creativity of the response (higher = more random)
    }
    try:
        response = requests.post(OLLAMA_URL, headers=headers, json=data)  # Send the POST request to Ollama
        if response.status_code == 200:  # If response is successful
            return response.json()['choices'][0]['message']['content']  # Return model's response
        else:
            return f"❌ Error: {response.status_code}, {response.text}"  # Handle errors if response is not OK
    except Exception as e:
        return f"❌ Error while contacting Ollama: {e}"  # Handle any other exceptions

# ========== File Extraction ==========
# Function to extract text from a .docx file
def extract_text_from_docx(file_path):
    try:
        doc = Document(file_path)  # Open .docx file
        return '\n'.join([para.text for para in doc.paragraphs])  # Extract text from all paragraphs
    except Exception as e:
        return f"❌ Error reading DOCX file: {e}"  # Return error if file cannot be read

# Function to extract text from a .pdf file
def extract_text_from_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:  # Open .pdf file in read-binary mode
            reader = PyPDF2.PdfReader(file)  # Create PDF reader object
            return '\n'.join([page.extract_text() for page in reader.pages if page.extract_text()])  # Extract text from each page
    except Exception as e:
        return f"❌ Error reading PDF file: {e}"  # Return error if file cannot be read

# ========== CV Analysis ==========
# Function to analyze the CV against a job description
def analyze_cv(cv_path, jd_path):
    # Check if Ollama is running before proceeding
    if not check_ollama_running():
        return "❌ Ollama is not running. Please start it first using 'ollama serve'."

    # Extract text from the CV file based on its extension (.docx or .pdf)
    if cv_path.endswith(".docx"):
        cv_text = extract_text_from_docx(cv_path)
    elif cv_path.endswith(".pdf"):
        cv_text = extract_text_from_pdf(cv_path)
    else:
        return "❌ Unsupported CV file format."

    # Extract text from the Job Description file based on its extension
    if jd_path.endswith(".docx"):
        jd_text = extract_text_from_docx(jd_path)
    elif jd_path.endswith(".pdf"):
        jd_text = extract_text_from_pdf(jd_path)
    else:
        return "❌ Unsupported job description file format."

    # Ensure that both CV and Job Description have text extracted successfully
    if not cv_text or not jd_text:
        return "❌ Failed to extract text from one or both files."

    # Create a prompt for Ollama to compare the CV with the job description
    prompt = f"""
    Compare the following resume with the job description:
    Resume: {cv_text}

    Job Description: {jd_text}

    Provide a similarity score (0-100), highlight key matches, and suggest areas to improve in the resume.
    """
    return ask_ollama(prompt)  # Send the prompt to Ollama for analysis

# ========== GUI Class ==========
class CVAnalyzerApp:
    # Initialize the GUI application
    def __init__(self, root):
        self.root = root
        self.root.title("📄 CVlyzer By Angad")  # Set window title
        self.root.geometry("800x650")  # Set window size
        self.style = tb.Style("darkly")  # Use a pre-defined theme from ttkbootstrap

        self.cv_path = tk.StringVar()  # Variable to store CV file path
        self.jd_path = tk.StringVar()  # Variable to store Job Description file path

        self.create_widgets()  # Call method to create widgets (UI elements)

    # Method to create all necessary UI components
    def create_widgets(self):
        padding = {'padx': 10, 'pady': 5}  # Define padding for widgets

        # Label and Entry widget for selecting CV file
        tb.Label(self.root, text="Select CV File (.docx/.pdf):", font=("Helvetica", 12)).pack(**padding)
        tb.Entry(self.root, textvariable=self.cv_path, width=80).pack()  # Input field to show selected CV file
        tb.Button(self.root, text="Browse CV", command=self.browse_cv, bootstyle="primary").pack(pady=5)  # Button to browse files

        # Label and Entry widget for selecting Job Description file
        tb.Label(self.root, text="Select Job Description File (.docx/.pdf):", font=("Helvetica", 12)).pack(**padding)
        tb.Entry(self.root, textvariable=self.jd_path, width=80).pack()  # Input field to show selected Job Description file
        tb.Button(self.root, text="Browse Job Description", command=self.browse_jd, bootstyle="primary").pack(pady=5)

        # Button to start the analysis
        tb.Button(self.root, text="🔍 Analyze", bootstyle="success", command=self.run_analysis).pack(pady=20)

        # Output area (scrollable text box) to display analysis results
        self.output = scrolledtext.ScrolledText(self.root, wrap=tk.WORD, font=("Courier", 11), height=20)
        self.output.pack(fill=tk.BOTH, padx=10, pady=10, expand=True)

    # Function to browse and select CV file
    def browse_cv(self):
        path = filedialog.askopenfilename(filetypes=[("Word or PDF", "*.docx *.pdf")])
        if path:
            self.cv_path.set(path)  # Update the path variable with the selected file

    # Function to browse and select Job Description file
    def browse_jd(self):
        path = filedialog.askopenfilename(filetypes=[("Word or PDF", "*.docx *.pdf")])
        if path:
            self.jd_path.set(path)  # Update the path variable with the selected file

    # Function to run the analysis after the user has selected both files
    def run_analysis(self):
        cv_file = self.cv_path.get()  # Get the CV file path
        jd_file = self.jd_path.get()  # Get the Job Description file path

        # Ensure both files have been selected
        if not cv_file or not jd_file:
            messagebox.showwarning("Missing Files", "Please select both CV and Job Description files.")
            return

        # Clear the output area and display a loading message
        self.output.delete("1.0", tk.END)
        self.output.insert(tk.END, "⏳ Analyzing... please wait...\n")
        self.root.update()  # Update the GUI

        # Perform the analysis by calling the analyze_cv function
        result = analyze_cv(cv_file, jd_file)

        # Display the analysis result in the output area
        self.output.delete("1.0", tk.END)
        self.output.insert(tk.END, result)

# ========== Launch App ==========
# Main execution entry point for the application
if __name__ == "__main__":
    root = tb.Window(themename="darkly")  # Create the root window with dark theme
    app = CVAnalyzerApp(root)  # Instantiate the CVAnalyzerApp
    root.mainloop()  # Start the Tkinter event loop to run the application
